In [52]:
import pandas as pd
data = pd.read_csv("pdb_dataset_with_label", sep=",")

In [53]:
data.shape

(5316, 24)

In [54]:
data.head()

,name,hydrophobicity,pocket_charge,volume,ALA,ARG,ASN,ASP,CYS,GLN,...,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL
0,10gs,-0.486000,0.020000,24.008835,0.000000,0.060000,0.080000,0.040000,0.06,0.060000,...,0.080000,0.060000,0.020000,0.020000,0.060000,0.020000,0.040000,0.020000,0.080000,0.060000
1,184l,0.541667,0.062500,16.115952,0.104167,0.062500,0.083333,0.020833,0.00,0.020833,...,0.125000,0.041667,0.062500,0.062500,0.020833,0.041667,0.020833,0.041667,0.020833,0.104167
2,185l,0.711905,0.071429,16.744284,0.142857,0.071429,0.047619,0.023810,0.00,0.023810,...,0.142857,0.047619,0.047619,0.071429,0.023810,0.047619,0.000000,0.047619,0.023810,0.119048
3,186l,0.516327,0.061224,15.958895,0.102041,0.061224,0.081633,0.020408,0.00,0.020408,...,0.122449,0.040816,0.061224,0.061224,0.020408,0.040816,0.040816,0.040816,0.020408,0.102041
4,187l,0.604545,0.068182,15.820103,0.113636,0.068182,0.045455,0.022727,0.00,0.022727,...,0.136364,0.045455,0.045455,0.068182,0.022727,0.045455,0.022727,0.045455,0.022727,0.113636


In [55]:
null_num = data.isnull().sum()
null_num_array = null_num.ravel()
null_num_sorted = sorted(null_num_array)

In [56]:
# Assuming null_num_sorted is the sorted array of counts of missing values

min_missing_count = min(null_num_sorted)  # Get the minimum count of missing values

min_missing_index = list(null_num_array).index(min_missing_count)  # Find the index of the minimum count

# Get the drug name corresponding to the minimum count
drug_name = data.columns[min_missing_index]

print("Drug with the least missing values:", drug_name)
print("Number of missing values:", min_missing_count)

Drug with the least missing values: name
Number of missing values: 0


In [57]:
pocket = 'name'

In [58]:
pocket_df = data[[pocket]]

In [59]:
# Cleaning expression data
data = data.dropna()
data = data.drop_duplicates()
print('data',data.shape)

# Cleaning drug sensitivity data
pocket_df = pocket_df.dropna(subset=[pocket])
pocket_df = pocket_df.drop_duplicates()
print('pocket_df',pocket_df.shape)

data (5316, 24)
pocket_df (5316, 1)


In [60]:
import numpy as np

# Assuming `pocket_df` has a column 'pocket_pdb' for pocket names 
# and the rest of the columns are features.
X_data = data.drop(columns=['name']).to_numpy()  # Extract features as NumPy array
Y_data = pocket_df['name'].to_numpy()  # Extract pocket PDB names

print('X_data:', X_data.shape)  # Shape of feature matrix
print('Y_data:', Y_data.shape)  # Shape of labels (pocket names)

X_data: (5316, 23)
Y_data: (5316,)


In [72]:
Y_data

array([   0,    1,    2, ..., 5313, 5314, 5315])

In [61]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Ensure pocket_pdb column exists
if 'name' not in pocket_df.columns:
    print("Column 'name' not found in pocket_df!")
else:
    # Encode PDB pocket names into numerical labels
    label_encoder = LabelEncoder()
    Y_data = label_encoder.fit_transform(pocket_df['name'])  # Convert names to integers

    print('Y_data:', Y_data.shape)
    print('Unique labels:', np.unique(Y_data))  # Check assigned labels


Y_data: (5316,)
Unique labels: [   0    1    2 ... 5313 5314 5315]


In [62]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 0

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data,Y_data, 
                                                    test_size =0.2, 
                                                    random_state=RANDOM_SEED, 
                                                    shuffle=True, 
                                                    )

In [64]:
# Setting predictor
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(class_weight='balanced',
                             random_state=RANDOM_SEED)

# Training (FIT) predictor
rfc.fit(X_train, Y_train)

RandomForestClassifier(class_weight='balanced', random_state=0)

In [65]:
y_pred_proba = rfc.predict_proba(X_test)[:, 1]

y_pred_proba

array([0., 0., 0., ..., 0., 0., 0.])

In [66]:
y_pred =  rfc.predict(X_test)

y_pred

array([1711,   23, 1641, ..., 4129, 3054,  688])

In [ ]:
rfc = RandomForestClassifier(random_state=RANDOM_SEED)

In [ ]:
# Training (FIT) predictor
rfc.fit(X_train, Y_train)

In [ ]:
pred = rfc.predict_proba(X_test)
y_pred_proba = pred[:,1]

pred_train =  rfc.predict_proba(X_train)
y_train_pred_proba = pred_train[:,1]

In [ ]:
hyp = {
    'n_estimators':150, # Try:100-500
    'max_depth':5, # Try: 5-10, None
  #  'min_samples_split': 3,
  #  'min_samples_leaf': 3, 
    'random_state':0, # DO NOT CHANGE
}

fit_and_eval(hyp, X_train, Y_train, X_test, Y_test)